# Doc2Vecの実行条件調査（part1）

- ラベル毎にユニークな教師データを設定（このためテストデータから学習セットの大半が失われます）


- 品詞を落とさないようにする


- パラメータを一部変更（DBOWを使用／feature数を調整）


accuracyは 0.954 と向上した様子です。

## (1) テストデータ／環境準備

In [1]:
'''
    テスト環境を準備するためのモジュールを使用します。
'''
import sys
import os
learning_dir = os.path.abspath("../../") #<--- donusagi-bot/learning
os.chdir(learning_dir)

if learning_dir not in sys.path:
    sys.path.append(learning_dir)

## (2) Doc2Vecの動作確認

### (2-1) コーパス生成

コーパス（単語が半角スペースで区切られた文字列）生成時、一部の品詞を落とすようにします。

（＝learning.core.nlang.Nlang クラスの仕様に従います）

In [2]:
import numpy as np

from learning.core.learn.learning_parameter import LearningParameter
from learning.core.datasource import Datasource

_bot_id = 9  # bot_id = 9はセプテーニ
attr = {
    'include_failed_data': False,
    'include_tag_vector': False,
    'classify_threshold': 0.5,
    'algorithm': LearningParameter.ALGORITHM_LOGISTIC_REGRESSION,
    'params_for_algorithm': {'C': 140},
    'excluded_labels_for_fitting': None
}

learning_parameter = LearningParameter(attr)

In [3]:
_datasource = Datasource(type='csv')
learning_training_messages = _datasource.learning_training_messages(_bot_id)
questions = np.array(learning_training_messages['question'])
answer_ids = np.array(learning_training_messages['answer_id'])

2017/05/18 PM 12:02:01 ['./fixtures/learning_training_messages/benefitone.csv', './fixtures/learning_training_messages/ptna.csv', './fixtures/learning_training_messages/septeni.csv', './fixtures/learning_training_messages/toyotsu_human.csv']
2017/05/18 PM 12:02:01 ['./fixtures/question_answers/toyotsu_human.csv']


In [4]:
import MeCab
import mojimoji

class Nlang_naive:
    @classmethod
    def split(self, text):
        tagger = MeCab.Tagger("-u learning/dict/custom.dic")
        tagger.parse('')  # node.surfaceを取得出来るようにするため、空文字をparseする(Python3のバグの模様)
        node = tagger.parseToNode(text)
        word_list = []
        while node:
            features = node.feature.split(",")
            pos = features[0]
            if pos in ["BOS/EOS", "記号"]:
                node = node.next
                continue

            #print(features)
            lemma = node.feature.split(",")[6]

            if lemma == "*":
                lemma = node.surface  #.decode("utf-8")
                
            word_list.append(mojimoji.han_to_zen(lemma))
            node = node.next
        return " ".join(word_list)

    @classmethod
    def batch_split(self, texts):
        splited_texts = []
        for text in texts:
            splited_texts.append(self.split(text))
        return splited_texts

In [5]:
from learning.core.nlang import Nlang

_sentences = np.array(questions)
_separated_sentences = Nlang_naive.batch_split(_sentences)

#### 試しに、学習セットをラベル毎にユニーク化してみます。

In [6]:
unique_answer_ids = []
unique_questions = []
unique_separated_sentences = []

for index, answer_id in enumerate(answer_ids):
    if answer_id not in unique_answer_ids:
        '''
            データをリストに格納
        '''
        unique_answer_ids.append(answer_id)
        unique_questions.append(questions[index])
        unique_separated_sentences.append(_separated_sentences[index])

In [7]:
'''
    教師データを表示
'''
for index, answer_id in enumerate(unique_answer_ids):
    print(answer_id, unique_separated_sentences[index])

4420 Ｗｉｎｄｏｗｓ ログイン が 出来る ない
2761 ブルース クリーン が 発生 する 強制 的 に シャット ダウン する れる
4423 パソコン に メモリ を 増設 する たい
4425 共有 ＰＣ に 設定 する て いる 管理 者 権限 を 他 の ユーザー に も 付与 する せる たい
4428 データ を メディア に コピー する たい
4429 Ｍａｃ を 使用 する たい
4432 誤る 送信 防止 システム の インストール が 出来る ない
4434 ａｋｉｎｄｏ に 接続 できる ない
4437 サイボウズ に ログイン できる ない
4441 販管 （ ＳＡＰ ） の パスワード を 忘れる た
4444 拠点 の ＩＰ アドレス が 知る たい
4445 プリンタードライバ の インストール が 出来る ない
4447 ＰＣ を 複数 台 使用 する たい
4450 Ｃ ドライブ の 容量 が 枯渇
4454 Ｃ ドライブ の 容量 が 不足
4455 複数 の メールアドレス の アカウント が 欲しい
4530 マウス が 壊れる た
4458 マウス 破損
4459 マウス 交換
4610 共有 フォルダ に アクセス できる ない
4460 共有 フォルダ アクセス
4468 リモート ＶＰＮ 接続 が 出来る ない
4472 リモート アクセス が 出来る ない
4473 ＶＰＮ 接続 が 出来る ない
4474 海外 出張 の 際 に ＰＣ を 持つ て いく たい
4478 ＭＬ が 届く ない
7084 ＰＣ が 破損 する ます た
4480 ＰＣ 破損
4484 ＰＣ が 不具合 を 起こす て いる ます
4489 Ｏｆｆｉｃｅ ファイル を ＰＤＦ 化 する たい
4491 ＩＤ システム 導入 前 の 事前 設定 ５ ＿ 販管 （ ＳＡＰ ） 利用 手順
4492 ＩＤ システム 導入 後 の 設定 Ｇ メール ログイン 手順 ． ｐｄｆ
4500 Ｇｍａｉｌ 手順
4493 無線 設定 する た のに （ ａｋｉｎｄｏ ｈｉｎｅｒａｎｋａｉ − ７ ） 繋がる ない
4494 ａｋｉｎｄｏ 繋がる ない
4495 ｈｉｎｅｒａｎｋａｉ − ７ 繋がる ない
4496 サイボウズ の パスワード

### (2-2) コーパスにタグ付け

models.doc2vecの仕様に従います。

In [8]:
from gensim import models
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

def doc_to_sentence(sentences, name):
    words = sentences.split(' ')
    return TaggedDocument(words=words, tags=[name])

def corpus_to_sentences(separated_sentences, answer_ids):
    for idx, (doc, name) in enumerate(zip(separated_sentences, answer_ids)):
        yield doc_to_sentence(doc, name)

### (2-3) 学習処理／モデルのシリアライズ

In [9]:
'''
    ユニークになったコーパス／ラベルから、
    学習セットを生成する
    
    教師データ件数＝173
'''
sentences = corpus_to_sentences(unique_separated_sentences, unique_answer_ids)
sentence_list = list(sentences)
len(sentence_list)

173

In [10]:
'''
    パラメータ変更点：
    　学習時にDBOWを使用する
    　featureの数をサンプル件数に近い値に設定
'''
model = Doc2Vec(dm=0, size=200, min_count=1, iter=1000)

In [11]:
'''
    ボキャブラリ生成／学習実行
    学習モデルは、ファイルに保存しておく
'''
model.build_vocab(sentence_list)
model.train(sentence_list)

model_path = 'prototype/better_algorithm/doc2vec.model'
model.save(model_path)

In [12]:
'''
    モデル内に保持されているベクトルの数を取得
    （教師データ数と同じであることを確認）
'''
len(model.docvecs)

173

In [13]:
model.docvecs

### (2-4) 予測処理

質問文は一部「揺れ」を与えたものとなっております

In [14]:
model_path = 'prototype/better_algorithm/doc2vec.model'

def predict(word, model_path):
    '''
        予測処理にかけるコーパスを生成
        （学習セット作成時と同じ関数を使用）
    '''
    corpus = Nlang_naive.split(word).split()

    '''
        コーパスからベクトルを生成し、
        ロードしたモデルから類似ベクトルを検索
    '''
    loaded_model = models.Doc2Vec.load(model_path)
    inferred_vector = loaded_model.infer_vector(corpus)
    ret = loaded_model.docvecs.most_similar([inferred_vector])

    return corpus, ret

In [15]:
'''
    マウスが破損（正解＝4458）
'''
predict('マウスが破損', model_path)

(['マウス', 'が', '破損'],
 [(4458, 0.8751524686813354),
  (4480, 0.8119498491287231),
  (4530, 0.7885047197341919),
  (4459, 0.7621862888336182),
  (7084, 0.7442445755004883),
  (4564, 0.7214786410331726),
  (4526, 0.6284592151641846),
  (4598, 0.6123510599136353),
  (4558, 0.6052687764167786),
  (4562, 0.5997931361198425)])

In [16]:
'''
    無線を使用したい（正解＝4516）
'''
predict('無線を使用したい', model_path)

(['無線', 'を', '使用', 'する', 'たい'],
 [(4516, 0.8372302055358887),
  (4521, 0.7616977095603943),
  (4429, 0.7409229278564453),
  (4577, 0.7279195785522461),
  (4447, 0.683928906917572),
  (4623, 0.6831488609313965),
  (4558, 0.6764004230499268),
  (4517, 0.6714218258857727),
  (4598, 0.6705586314201355),
  (4592, 0.6627346277236938)])

In [17]:
'''
    情報システムのアドレス（正解＝7040）
'''
predict('情報システムのアドレス', model_path)

(['情報', 'システム', 'の', 'アドレス'],
 [(7065, 0.7541380524635315),
  (7040, 0.7414141893386841),
  (4444, 0.7272987365722656),
  (4547, 0.7148098349571228),
  (4539, 0.7023628950119019),
  (7064, 0.6538846492767334),
  (4562, 0.6430391073226929),
  (4597, 0.6365411281585693),
  (4531, 0.629902720451355),
  (4598, 0.6289425492286682)])

In [18]:
'''
    Office2010を使用したいのですが（正解＝4625）
'''
predict('Office2010を使用したいのですが', model_path)

(['Ｏｆｆｉｃｅ', '２０１０', 'を', '使用', 'する', 'たい', 'の', 'です', 'が'],
 [(4558, 0.7349570989608765),
  (4625, 0.7241668105125427),
  (4523, 0.706867516040802),
  (4598, 0.6768537759780884),
  (4489, 0.6751973628997803),
  (4562, 0.667708158493042),
  (4590, 0.6624493598937988),
  (4533, 0.6622073650360107),
  (4531, 0.6591483354568481),
  (4602, 0.6417791247367859)])

In [19]:
'''
    携帯からサイボウズを使いたいのですが、どうしたら出来ますか？
    （正解＝4504だが、学習時の教師データから抜け落ちたもの）
'''
predict('携帯からサイボウズを使いたいのですが、どうしたら出来ますか？', model_path)

(['携帯',
  'から',
  'サイボウズ',
  'を',
  '使う',
  'たい',
  'の',
  'です',
  'が',
  'どう',
  'する',
  'た',
  '出来る',
  'ます',
  'か'],
 [(7084, 0.6905070543289185),
  (4558, 0.6836915016174316),
  (4480, 0.6532915234565735),
  (4501, 0.6294933557510376),
  (4541, 0.6261754035949707),
  (4557, 0.6092617511749268),
  (4602, 0.5909730792045593),
  (4458, 0.5903823375701904),
  (4533, 0.5843251943588257),
  (7042, 0.5839012265205383)])

## (3) accuracy 測定

ここからが本題です

In [20]:
model_path = 'prototype/better_algorithm/doc2vec.model'

len(unique_separated_sentences)

173

In [21]:
def predict_similarity(separated_sentence, model_path):
    corpus = separated_sentence.split()
    loaded_model = models.Doc2Vec.load(model_path)
    inferred_vector = loaded_model.infer_vector(corpus)
    ret = loaded_model.docvecs.most_similar([inferred_vector])

    answer_id, similarity = ret[0]
    return corpus, answer_id, similarity

In [22]:
def get_prediction_statistics(separated_sentences, answer_ids, model_path):
    '''
        学習セットの質問文をそのまま予測処理にかけて、
        回答を予測
    '''
    statistics = []
    for i, _ in enumerate(separated_sentences):
        sentence = separated_sentences[i]
        preferred_answer_id = answer_ids[i]
        corpus, answer_id, similarity = predict_similarity(separated_sentences[i], model_path)
        corpus_len = len(corpus)
        statistics.append((i, corpus_len, preferred_answer_id, answer_id, similarity))

    return statistics

In [23]:
prediction_statistics = get_prediction_statistics(unique_separated_sentences, unique_answer_ids, model_path)

In [24]:
ncorrect_by_corpus_len = {}
nsample_by_corpus_len = {}

ncorrect_by_answer_id = {}
nsample_by_answer_id = {}
corpus_len_by_answer_id = {}

ncorrect = 0
nsample = 0

'''
    予測結果を、質問文の単語数毎／回答ID毎に統計する
'''
for statistics in prediction_statistics:
    i, corpus_len, preferred_answer_id, answer_id, similarity = statistics
    
    '''
        質問文の単語数ごとに統計を取る
    '''
    if corpus_len not in nsample_by_corpus_len.keys():
        ncorrect_by_corpus_len[corpus_len] = 0
        nsample_by_corpus_len[corpus_len] = 0
    nsample_by_corpus_len[corpus_len] += 1

    '''
        回答IDごとに統計を取る
    '''
    if preferred_answer_id not in nsample_by_answer_id.keys():
        ncorrect_by_answer_id[preferred_answer_id] = 0
        nsample_by_answer_id[preferred_answer_id] = 0
        corpus_len_by_answer_id[preferred_answer_id] = 0
    nsample_by_answer_id[preferred_answer_id] += 1
    corpus_len_by_answer_id[preferred_answer_id] += corpus_len
    
    '''
        正解かどうか検査
    '''
    nsample += 1
    if preferred_answer_id == answer_id:
        ncorrect += 1
        ncorrect_by_corpus_len[corpus_len] += 1
        ncorrect_by_answer_id[preferred_answer_id] += 1

In [25]:
'''
    質問文の単語数ごとの統計情報を編集
'''
info_by_corpus_len = []
for k, v in ncorrect_by_corpus_len.items():
    info_by_corpus_len.append((
        k, 
        ncorrect_by_corpus_len[k]/nsample_by_corpus_len[k], 
        ncorrect_by_corpus_len[k], 
        nsample_by_corpus_len[k]
    ))

In [26]:
'''
    回答IDごとの統計情報を編集
'''
info_by_answer_id = []
for k, v in ncorrect_by_answer_id.items():
    info_by_answer_id.append((
        k, 
        ncorrect_by_answer_id[k]/nsample_by_answer_id[k], 
        ncorrect_by_answer_id[k], 
        nsample_by_answer_id[k],
        corpus_len_by_answer_id[k]/nsample_by_answer_id[k]
    ))

In [27]:
'''
    全体の正解率
'''
print("accuracy=%0.3f (%d/%d)" % (
    ncorrect/nsample, ncorrect, nsample))

accuracy=0.954 (165/173)


In [28]:
'''
    質問文の単語数ごとの正解率をリスト
'''
for info in info_by_corpus_len:
    print("word_count=%2d: accuracy=%0.3f (%d/%d)" % (
        info[0], info[1], info[2], info[3]
    ))

word_count= 1: accuracy=1.000 (1/1)
word_count= 2: accuracy=0.857 (6/7)
word_count= 3: accuracy=0.833 (5/6)
word_count= 4: accuracy=1.000 (10/10)
word_count= 5: accuracy=0.833 (15/18)
word_count= 6: accuracy=0.857 (18/21)
word_count= 7: accuracy=1.000 (20/20)
word_count= 8: accuracy=1.000 (20/20)
word_count= 9: accuracy=1.000 (12/12)
word_count=10: accuracy=1.000 (7/7)
word_count=11: accuracy=1.000 (10/10)
word_count=12: accuracy=1.000 (11/11)
word_count=13: accuracy=1.000 (11/11)
word_count=14: accuracy=1.000 (5/5)
word_count=15: accuracy=1.000 (6/6)
word_count=16: accuracy=1.000 (2/2)
word_count=17: accuracy=1.000 (2/2)
word_count=19: accuracy=1.000 (1/1)
word_count=20: accuracy=1.000 (1/1)
word_count=22: accuracy=1.000 (1/1)
word_count=30: accuracy=1.000 (1/1)


In [29]:
'''
    正解でなかったラベルの一覧。

    参考情報として、回答ラベルがつけられている
    質問文の単語数（word count）を表示
'''
sorted_info_by_answer_id = sorted(info_by_answer_id, key=lambda x:x[1], reverse=False)
for info in sorted_info_by_answer_id:
    if info[1] > 0:
        continue #当たりは表示しない
    print("answer_id=%4d: word count=%2d" % (
        info[0], info[4]
    ))

answer_id=4434: word count= 5
answer_id=4437: word count= 5
answer_id=4454: word count= 6
answer_id=4468: word count= 6
answer_id=7037: word count= 2
answer_id=4506: word count= 5
answer_id=4515: word count= 6
answer_id=4600: word count= 3
